# Введение в MapReduce модель на Python


In [183]:

from typing import Iterator
from typing import NamedTuple  # requires python 3.6+

In [184]:
def MAP(_, row: NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)


def REDUCE(age: str, rows: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum / count)
    else:
        yield (age, 0)

Модель элемента данных

In [185]:
class User(NamedTuple):
    id: int
    age: int
    social_contacts: int
    gender: str

In [186]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [187]:
def RECORDREADER():
    return [(u.id, u) for u in input_collection]

In [188]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [189]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

In [190]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output)  # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [191]:
def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

In [192]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [193]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [194]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [195]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [196]:
from typing import NamedTuple  # requires python 3.6+
from typing import Iterator


class User(NamedTuple):
    id: int
    age: str
    social_contacts: int
    gender: str


input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]


def MAP(_, row: NamedTuple):
    if (row.gender == 'female'):
        yield (row.age, row)


def REDUCE(age: str, rows: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for row in rows:
        sum += row.social_contacts
        count += 1
    if (count > 0):
        yield (age, sum / count)
    else:
        yield (age, 0)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [197]:
from typing import Iterator
import numpy as np

mat = np.ones((5, 4))
vec = np.random.rand(4)  # in-memory vector in all map tasks


def MAP(coordinates: (int, int), value: int):
    i, j = coordinates
    yield (i, value * vec[j])


def REDUCE(i: int, products: Iterator[NamedTuple]):
    sum = 0
    for p in products:
        sum += p
    yield (i, sum)


def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i, j])


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.519736733898636)),
 (1, np.float64(2.519736733898636)),
 (2, np.float64(2.519736733898636)),
 (3, np.float64(2.519736733898636)),
 (4, np.float64(2.519736733898636))]

## Inverted index

In [198]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]


def RECORDREADER():
    for (docid, document) in enumerate(documents):
        yield ("{}".format(docid), document)


def MAP(docId: str, body: str):
    for word in set(body.split(' ')):
        yield (word, docId)


def REDUCE(word: str, docIds: Iterator[str]):
    yield (word, sorted(docIds))


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [199]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]


def RECORDREADER():
    for (docid, document) in enumerate(documents):
        for (lineid, line) in enumerate(document.split('\n')):
            yield ("{}:{}".format(docid, lineid), line)


def MAP(docId: str, line: str):
    for word in line.split(" "):
        yield (word, 1)


def REDUCE(word: str, counts: Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [200]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in
            enumerate(partitions)]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(lambda reduce_partition: (
        reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))),
                         reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(
        sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [201]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for (lineid, line) in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid, lineid), line)

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i + split_size])


def MAP(docId: str, line: str):
    for word in line.split(" "):
        yield (word, 1)


def REDUCE(word: str, counts: Iterator[int]):
    sum = 0
    for c in counts:
        sum += c
    yield (word, sum)


# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2), ('is', 18)]),
 (1, [('a', 2), ('it', 18), ('what', 10)])]

## TeraSort

In [202]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for value in split:
            yield (value, None)

    split_size = int(np.ceil(len(input_values) / maps))
    for i in range(0, len(input_values), split_size):
        yield RECORDREADER(input_values[i:i + split_size])


def MAP(value: int, _):
    yield (value, None)


def PARTITIONER(key):
    global reducers
    global max_value
    global min_value
    bucket_size = (max_value - min_value) / reducers
    bucket_id = 0
    while ((key > (bucket_id + 1) * bucket_size) and ((bucket_id + 1) * bucket_size < max_value)):
        bucket_id += 1
    return bucket_id


def REDUCE(value: int, _):
    yield (None, value)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.01870573900378536)),
   (None, np.float64(0.024408245440828202)),
   (None, np.float64(0.025418629463344278)),
   (None, np.float64(0.1658236271032446)),
   (None, np.float64(0.16694573982404026)),
   (None, np.float64(0.20208720582999562)),
   (None, np.float64(0.2058916435167838)),
   (None, np.float64(0.21122888443167342)),
   (None, np.float64(0.30321616821416697)),
   (None, np.float64(0.3671866999172657)),
   (None, np.float64(0.376242817512791)),
   (None, np.float64(0.3943836079915053)),
   (None, np.float64(0.44859323525266703)),
   (None, np.float64(0.46655696611010955))]),
 (1,
  [(None, np.float64(0.5771027516890415)),
   (None, np.float64(0.5924131375951919)),
   (None, np.float64(0.6085228841809663)),
   (None, np.float64(0.6945950281795732)),
   (None, np.float64(0.7413923653852651)),
   (None, np.float64(0.750986119855691)),
   (None, np.float64(0.7571583333405224)),
   (None, np.float64(0.7580318234022124)),
   (None, np.float64(0.7613540446

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [203]:
def MAP(_, row: NamedTuple):
    yield 'max', row


def REDUCE(aggregate: str, rows: list[NamedTuple]):
    max_value = rows[0]
    for row in rows:
        if row.social_contacts > max_value.social_contacts:
            max_value = row
    yield aggregate, max_value.social_contacts


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('max', 800)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [204]:
def MAP(_, row: NamedTuple):
    yield 'avg', row


def REDUCE(aggregate: str, rows: list[NamedTuple]):
    c = 0
    s = 0
    for row in rows:
        s += row.social_contacts
        c += 1
    yield aggregate, s / c


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('avg', 390.0)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [205]:
def groupbykey(iterable):
    # sort by social_contacts
    iterable = sorted(iterable, key=lambda user: user[1].social_contacts)
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('avg', 390.0)]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [211]:
input_collection = [
    User(id=0, age=17, gender='male', social_contacts=6146),
    User(id=0, age=13, gender='female', social_contacts=223),
    User(id=1, age=24, gender='male', social_contacts=6363),
    User(id=2, age=51, gender='female', social_contacts=124),
    User(id=2, age=91, gender='male', social_contacts=666),
    User(id=3, age=45, gender='female', social_contacts=12416),
    User(id=3, age=81, gender='female', social_contacts=1512),
    User(id=4, age=11, gender='male', social_contacts=81),
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [
        (partition_id, sorted(partition.items(), key=lambda x: x[0]))
        for (partition_id, partition) in enumerate(partitions)
    ]


def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers


def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER != None:
        map_partitions = map(
            lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle
    reduce_outputs = map(lambda reduce_partition: (
        reduce_partition[0],
        flatten(map(
            lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]
        ))
    ), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(
        sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs


maps = 3
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i: i + split_size])


def MAP_UNIQUE(_, user: str):
    yield (user.id, user)


def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]


partitioned_output = MapReduceDistributed(
    INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None
)
partitioned_output = [
    (partition_id, list(partition)) for (partition_id, partition) in partitioned_output
]
partitioned_output

8 key-value pairs were sent over a network.


[(0,
  [User(id=0, age=17, social_contacts=6146, gender='male'),
   User(id=2, age=51, social_contacts=124, gender='female'),
   User(id=4, age=11, social_contacts=81, gender='male')]),
 (1,
  [User(id=1, age=24, social_contacts=6363, gender='male'),
   User(id=3, age=45, social_contacts=12416, gender='female')])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [212]:
def MAP(_, row: NamedTuple):
    if row.age == 11:
        yield (row, row)


def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=4, age=11, social_contacts=81, gender='male'),
  [User(id=4, age=11, social_contacts=81, gender='male')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [243]:
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )


def MAP(_, row: NamedTuple):
    S = ['age', 'social_contacts']
    projection = tuple((s, getattr(row, s)) for s in S)
    yield projection, projection


def REDUCE(row: str, rows: Iterator[NamedTuple]):
    yield rows


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[(('age', 17), ('social_contacts', 6146))],
 [(('age', 13), ('social_contacts', 223))],
 [(('age', 24), ('social_contacts', 6363))],
 [(('age', 51), ('social_contacts', 124))],
 [(('age', 91), ('social_contacts', 666))],
 [(('age', 45), ('social_contacts', 12416))],
 [(('age', 81), ('social_contacts', 1512))],
 [(('age', 11), ('social_contacts', 81))]]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [250]:
from typing import Iterator

input_collection_a = [
    User(id=1, age=17, gender='male', social_contacts=51),
    User(id=2, age=13, gender='female', social_contacts=25),
    User(id=3, age=24, gender='male', social_contacts=424),
    User(id=4, age=51, gender='male', social_contacts=455),
    User(id=5, age=91, gender='female', social_contacts=152),
]

input_collection_b = [
    User(id=4, age=51, gender='male', social_contacts=455),
    User(id=5, age=91, gender='female', social_contacts=152),
    User(id=6, age=23, gender='female', social_contacts=44),
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


def MAP(_, row: NamedTuple):
    yield row.id, row


def REDUCEN(row: str, rows: Iterator[NamedTuple]):
    for elem in rows:
        yield row, elem


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[1, 2, 3, 4, 5, 6]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [246]:
def MAP(_, row: NamedTuple):
    yield row.id, row


def REDUCE(row: str, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows[0], rows[1]


def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=4, age=51, social_contacts=455, gender='male'),
  User(id=4, age=51, social_contacts=666, gender='female')),
 (User(id=5, age=91, social_contacts=152, gender='female'),
  User(id=5, age=44, social_contacts=125, gender='male'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [251]:
def MAP(collection_id, user):
    yield user, collection_id


def REDUCE(user, collections):
    if 1 not in collections:
        yield user


# На первом месте стоит номер выборки
def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[User(id=1, age=17, social_contacts=51, gender='male'),
 User(id=2, age=13, social_contacts=25, gender='female'),
 User(id=3, age=24, social_contacts=424, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [254]:
class User(NamedTuple):
    id: int
    age: int
    social_contacts: int
    gender: str
    address_id: int


users_collection = [
    User(id=1, age=17, gender='male', social_contacts=51, address_id=1),
    User(id=2, age=13, gender='female', social_contacts=25, address_id=1),
    User(id=3, age=24, gender='male', social_contacts=424, address_id=3),
    User(id=4, age=51, gender='male', social_contacts=455, address_id=1),
    User(id=5, age=91, gender='female', social_contacts=152, address_id=4),
]


class Address(NamedTuple):
    id: int
    line: str


addresses_collection = [
    Address(id=1, line="1"),
    Address(id=2, line="2"),
    Address(id=3, line="3"),
    Address(id=4, line="4"),
    Address(id=5, line="5")
]


def MAP(address_id, row):
    yield address_id, row


def REDUCE(address_id, rows):
    users = []
    address = None

    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            address = row

    for row in rows:
        if type(row) is User:
            yield row, row.address_id, address


def RECORDREADER():
    return [(user.address_id, user) for user in users_collection] + [(address.id, address) for address in
                                                                     addresses_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
join = output
output

[(User(id=1, age=17, social_contacts=51, gender='male', address_id=1),
  1,
  Address(id=1, line='1')),
 (User(id=2, age=13, social_contacts=25, gender='female', address_id=1),
  1,
  Address(id=1, line='1')),
 (User(id=4, age=51, social_contacts=455, gender='male', address_id=1),
  1,
  Address(id=1, line='1')),
 (User(id=3, age=24, social_contacts=424, gender='male', address_id=3),
  3,
  Address(id=3, line='3')),
 (User(id=5, age=91, social_contacts=152, gender='female', address_id=4),
  4,
  Address(id=4, line='4'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [256]:
def MAP(address_id, user, address):
    yield address_id, user


def REDUCE(address_id, rows):
    yield f'address with id={address_id} has {len(rows)} users'


def RECORDREADER():
    return [(address_id, user, address) for user, address_id, address in join]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

['address with id=1 has 3 users',
 'address with id=3 has 1 users',
 'address with id=4 has 1 users']

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [257]:
from typing import Iterator

input_matrix = [
    (4, 2, 1), (0, 6, 3), (1, 2, 4),
    (2, 6, 4), (2, 7, 2), (4, 6, 5),
    (2, 1, 0), (4, 1, 5), (2, 1, 3),
]

input_vector = [
    (2, 1), (1, 2), (0, 2)
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    grouped = {}
    for key, value in iterable:
        grouped[key] = grouped.get(key, []) + [value]
    return grouped.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(
        map(
            lambda x: REDUCE(*x),
            groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))),
        )
    )


def MAP(_, matrix_row):
    row, col, value = matrix_row
    for vec_col, vec_value in input_vector:
        if vec_col == col:
            yield row, value * vec_value


def REDUCE(row, values: Iterator[int]):
    yield row, sum(values)


def RECORDREADER():
    return [(None, m) for m in input_matrix]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(4, 11), (1, 4), (2, 6)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [258]:
# MapReduce model
def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
    return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [259]:
import numpy as np

I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)  # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J, K)


def RECORDREADER():
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            yield ((j, k), big_mat[j, k])


def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        yield (i, k), small_mat[i, j] * w


def REDUCE(key, values):
    (i, k) = key
    yield (i, k), sum(values)

Проверьте своё решение

In [260]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))  # should return true

True

In [261]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i, k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [262]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)


def MAP(m1, m2):
    (i, j), v1 = m1
    (j, k), v2 = m2
    yield (i, k), v1 * v2


def REDUCE(key, values):
    (i, k) = key
    yield (i, k), sum(values)


def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield ((i, j), small_mat[i, j]), ((j, k), big_mat[j, k])


reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [263]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps

    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield ((i, j), small_mat[i, j]), ((j, k), big_mat[j, k])

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


def MAP(m1, m2):
    (i, j), v1 = m1
    (j, k), v2 = m2
    yield (i, k), v1 * v2


def REDUCE(key, values):
    (i, k) = key
    yield (i, k), sum(values)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))  # should return true

240 key-value pairs were sent over a network.
[[0.79561997 1.76493894 0.64161973 0.5041523  1.00149622 1.36946019
  1.6657596  1.67684697 1.29678977 1.26179231 0.99762795 1.63638186
  0.80223519 1.26389222 0.59498962 0.30654319 0.24800222 1.43435657
  0.77081349 0.89021331 0.85760739 1.06014117 1.28494319 1.77366639
  0.9101468  0.73594699 1.51365778 1.00520778 0.82159279 0.65528844
  1.07558216 0.97900933 1.36028928 1.36315073 1.33392974 0.57602444
  1.40573241 1.74023166 1.32307561 1.45752511]
 [0.57458361 1.37150452 0.58469603 0.36235124 0.58593613 1.03700091
  1.29073957 1.15821754 0.99101005 0.96590285 0.72813583 1.13929648
  0.5384187  0.9443644  0.47647917 0.23232637 0.21749929 0.84788995
  0.81540592 0.6603552  0.76859747 0.71603044 1.11733862 1.39935156
  0.7554922  0.69028795 1.24364525 0.92388323 0.55132685 0.42455825
  0.77442997 0.76942484 1.08462813 0.96732408 1.23670656 0.55486952
  0.99184442 1.4346397  1.05144771 1.20605221]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
Да, так как генерируемые элементы идентифицируются индексами матрицы